# In this notebook you will learn how to use build SVM for binary classification and how to use quadratic optimization package CVXOPT in python.

Side Note. We don't have time to go into details about following things but I want you to be aware of following important usage of svm in practice.
- SVM supports regression too. Look into literature for regression usage
- SVM supports multiclass classification too
- We can combine muliple kernel to use data from different sources(video, image ,audio) and using right kernel to measure similarity and then combine.  See MKL https://en.wikipedia.org/wiki/Multiple_kernel_learning 
  
- **One class SVM** are used for novelty (anamolies, outlier, noise etc) detection in fraud detection, medical abnormality, production abnormality etc. One can also use **ensemble.IsolationForest** from ensemble methods.
- See this paper for a review of novelty detection methods. http://www.robots.ox.ac.uk/~davidc/pubs/NDreview2014.pdf

# Let's review some theory. Following formulation of SVM uses Dual formualtion of the problem. See following link to see the derivation. You need to understand  optimization theory too.

In this notebook you will implement  SVM. We know that SMV primal objective is

$\min_{w, w_0} \frac{1}{2}||w||^2 + C \sum_{i=1}^{N} \xi_i$ st $y_i(w^T +w_0) \gt 1 $  for $\forall i$

Note the label is $(+1, -1)$ instead of $(0, 1)$


Infact Dual problem(using Lagrangian) is written as 
$\min_{\alpha} \frac{1}{2}\sum_{i=1}^N \alpha_i \alpha_j y_i y_j(x_i^T x_j) - 1^T \alpha$ s.t $ \alpha_i \ge 0$ and $y^T\alpha = 0$

or 

$\min_{\alpha} \frac{1}{2}\alpha^T K \alpha - 1^T \alpha$ s.t $ \alpha_i \ge 0$ and $y^T\alpha = 0$

where  $\frac{1}{2}\alpha^T K \alpha$ = $\sum_{i=1}^N \alpha_i \alpha_j y_i y_j(x_i^T x_j)$ and $\alpha$ is vector of $\alpha_i$. Similar interpretation for $y$ and $y_i$. 


If you review optimization theory, you can look at the derivation here
http://cs229.stanford.edu/notes/cs229-notes3.pdf




dual problem depends only on innner product $x_i^Tx_j$ between $x_i, x_j$ . Hence we can replace it with a mercer kernal $\mathcal{k}(x_i, x_j) = \phi(x_i)^T \phi(x_j)$ where $\phi$ is the function
guranteed by mercer theorem. You can think of it as a feature mapping function $x_i \rightarrow \phi(x_i)$.
For kernel svm replace matrix K  using RBF kernel i.e $K_{ij} = y_i  y_j \mathcal{k}(x_i,x_j)$ where  $\mathcal{k}$ is any kernel. **Use RBF kernel for this assignment.**

Complete dual problem in dual form is
$\min_{\alpha} \frac{1}{2}\sum_{i=1}^N \alpha_i \alpha_j y_i y_j \mathcal{k}(x_i, x_j) - 1^T \alpha$ s.t $ \alpha_i \ge 0$ and $y^T\alpha = 0$




We will use cvxopt Quadratic program solver (QP).

cvxopt.solvers.qp(P, q[, G, h[, A, b[, solver[, initvals]]]])

which solves the problem


$\min_{x} \frac{1}{2}x^T P x {\bf+} q^T x$ s.t $Gx \preceq h$ and $Ax = b$. Note $\preceq$ denotes componentwise inequality


Hence we need to map to this interface. Clearly $x$ is $\alpha$ and $K$ is $P$, $q$ is vector of -1.

Now we have to build  inequality

**(i)** $\alpha_i \ge 0$ or $ -\alpha_i < 0 $ for all $i$ or in matrix 
form $- I\alpha \preceq {\bf 0}$ where $I$ is  identity matrix of size $N \times N$ and  ${\bf 0}$ is vector of zeros of size N(number of samples)




 
Hence $G $ will be $\begin{bmatrix} -I  \end{bmatrix}$  and $h$ is $\begin{bmatrix} {\bf 0}   \end{bmatrix} $


$A$  is $y^T$ and $b$ is $0$

Once you solve for $\alpha$, w is equal to $\sum_{i} \alpha_i y_i \phi(x_i)$ and $w_0$  is equal to $\frac{max_{j:y_j =-1} \sum_{i} \alpha_i y_i k(x_i, x_j) +  min_{j:y_j =1} \sum_{i} \alpha_i y_i k(x_i, x_j)}{2}$ 

use the sign$(w^T x_{test} + w_0)$  or  sign$(\sum_{i}^{N} \alpha_i  y_i \mathcal{k}(x_i,x_{test}) + w_0)$ to predict label of test data



# Let's install  cvxopt package for optimization 


In [ ]:
import os


<font color ='red'> Please run following cell only once.  </font>

In [ ]:
os.system('conda install cvxopt')
# You can run the command inside string from command prompt too


In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import cvxopt

 Click here to learn more about [Python Software for Convex Optimization](http://cvxopt.org/)

In [ ]:
data = load_breast_cancer()

In [ ]:
list(data.target_names)

In [ ]:
len(data.feature_names)

In [ ]:
X = data.data
y= data.target

In [ ]:

y[0:30]

## Let's convert dataset label to {-1, +1}
We need to convert 0 to -1 only

In [ ]:
y1= np.where(y==0, -1, y)

In [ ]:
y1[0:30]

In [ ]:
sum(y1 ==1), sum(y1 ==-1)

## We are ignoring data imbalance issue

In [ ]:
y = y1

In [ ]:
np.max(X, axis=0)

# Let's standardize the features

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X1 = scaler.transform(X)

In [ ]:
np.max(X1, axis=0)

In [ ]:
X= X1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [ ]:
y_train.shape, y_val_test.shape

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=2)

In [ ]:
y_val.shape, y_test.shape

# We will Implement Kernel SVM using solver from cvxopt. Use validation set to select best model and  evaluate the model on test set


In [ ]:
def  rbf_kernel (x1,x2, sigma):
    return np.exp(- np.linalg.norm(x1- x2)**2/(2.0* sigma**2))

In [ ]:
# Let declare matrix K
# We need to build it different sigma
K = np.zeros((X_train.shape[0], X_train.shape[0]))
print(X_train.shape, K.shape)
n_samples = X_train.shape[0]

In [ ]:
q= cvxopt.matrix(-1*np.ones(n_samples))
q.size

# Q1 (1 point) build matrix  G

In [ ]:
# -I 
G = ## Write code here
G.size

# Q2 (1 point) build matrix h

In [ ]:
h = ## wrire code here
h.size


## Note y_train contains integer label value (1, -1) but in cvopt library A needs to be double.

In [ ]:
A = cvxopt.matrix(np.expand_dims(y_train, axis=0), tc = 'd')
A.size

In [ ]:
b= cvxopt.matrix(0.0)

In [ ]:
def predict_label(x, svec_alphas, svecs, svec_labels, wo, sigma):
    val = wo
    for alpha, svec, lbl in  zip(svec_alphas, svecs, svec_labels):
        val +=  alpha*lbl*rbf_kernel(x,svec, sigma )
        
    return np.sign(val)   



In RBF kernel $k(x_1, x_1) = \exp(\frac{\|x_1 -x_2\|^2}{-2 \sigma^2})$. Also written as  $ \exp(- \gamma \|x_1 -x_2\|^2)$ there is a hyper parameter $\sigma$ which we need to decide before solving the  problem using optimizer.

- For low value of $\sigma$ or large value of $\gamma$, area of influence is close to any support vectors. Clearly support vector influence increases as $\sigma$ increase or $\gamma$ decreases. It is similary to effect of $\sigma$ in gussian distribution.



Search for $\sigma.$ 

<font color = 'red'>Notice choose right value of $\sigma$ is very important in 
RBF kernel. </font>



In [ ]:
SUPPORT_VECTOR_WEIGHT_THR = 1e-4

best_val_accuracy = -np.inf
best_support_vectors_info= None
for sigma in np.linspace(.05, 15, 15):    
    #Let's iterate over sample and fill matrix K
    # Maybe we can do following operations in matrix form.
    # Going with two loops right now
    for i, x1 in enumerate(X_train):
        for j, x2 in enumerate(X_train):
            #print(x1.shape, x2.shape)
            K[i,j] = rbf_kernel(x1, x2, sigma)
    # Write code here
    P = cvxopt.matrix(np.outer(y_train, y_train)*K )           
    # Let'solve the optimization problem using qudratic solver from cvxopt
    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    svec_loc = alphas > SUPPORT_VECTOR_WEIGHT_THR
    svec_alphas = alphas[svec_loc]
    
    # svecs contains our support vector
    svecs, svec_labels = X_train[ np.ravel(svec_loc),:], y_train[ np.ravel(svec_loc)]
    # evaluating the intercept for all the support vectors.
    svec_wo_pos = []
    svec_wo_neg = []
    for svec1 ,lbl1 in zip(svecs, svec_labels):
        val = 0
        for alpha, svec2, lbl2 in  zip(svec_alphas, svecs, svec_labels):
            val +=  alpha*lbl2*rbf_kernel(svec1,svec2, sigma )
        if lbl1== +1:    
            svec_wo_pos.append( val)
        elif lbl1 == -1:
            svec_wo_neg.append(val)
        else:
            print('erro support vector label in not +ve or -ve')

    #Let use first/ or mean?
    #wo = np.mean(svec_wo)
    wo = -(max(svec_wo_neg) + min(svec_wo_pos))/2.0
    
    val_acc = np.mean([ y == predict_label(x, svec_alphas, svecs, svec_labels, wo, sigma)
                       for  x, y in  zip(X_val, y_val)])
    print('sigma = {} validation accuracy = {}'.format(sigma,val_acc))
    if val_acc > best_val_accuracy:
        print('updating best model current val {} previous val {}'.format(val_acc, best_val_accuracy))
        best_support_vector_info = (sigma, svec_alphas,svecs, svec_labels, wo)
        best_val_accuracy = val_acc
        


In [ ]:

# picking best sigma for accuracy on validation set 

best_sigma, svec_alphas, svecs, svec_labels, wo = best_support_vector_info


# Q 3(2 point) write code  to evalue validation and test set accuracy

In [ ]:
# write code here

# Q4(1 point). Use sklearn kernel SVM with soft margin interface. Select best model and evaluate accuracy on test set using SVM from  the library

Check your accuracy value matches

Hint

- You need to use large value of C to simulate hard margin SVM
- gamma= 1/(2*best_sigma**2)

In [ ]:
from sklearn import svm
clf = ## write code here

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.intercept_

In [ ]:
wo